<a href="https://colab.research.google.com/github/Fefke/interactive-uniscenes/blob/main/ComfyUI_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI-3D

  - [How to](https://comfyui-wiki.com/en/install/install-comfyui/run-comfyui-on-cloud) install ComfyUI on Colab.
    - [Original](https://github.com/comfyanonymous/ComfyUI/blob/master/notebooks/comfyui_colab.ipynb) Jupyter-Template.
  -  Solve dependency conflicts? (`pip install pipdeptree`)
  

In [ ]:
# If you want to persist data, you have to store it on g-drive or github.
# ..unfortunately you have to connect to g-drive manually (once every session).
#@title Environment Setup



OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
WORKSPACE = "/content/drive/MyDrive/Colab-4-Studienarbeit/" #@param {type:"string"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

!mkdir -p $WORKSPACE
%cd $WORKSPACE

![ ! -d $WORKSPACE"/ComfyUI" ] && echo -= Initial setup ComfyUI =- && git clone --depth 1 https://github.com/comfyanonymous/ComfyUI
%cd ComfyUI

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!export PYTHONWARNINGS=always
!pip install --upgrade pip

  #? to make sure important warnings/errors are prompted, but be less verbose with --quiet
!echo -= Install dependencies =-
!pip install xformers==0.0.29 --quiet -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121
# 0.0.29.post3 not supported yet, as torch==2.6.0 is blocked by fastai

# I had to add the following..
!echo -= Resolve dependency issue: fastai requires torch<2.6 =-
# !pip uninstall torch torchvision torchaudio --quiet -y
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --quiet
# --force-reinstall

!echo -= Resolve other dependency issue =-
!pip install numpy==1.26.4 gcsfs==2025.2.0 jedi>=0.16 decorator==4.4.2 google-cloud-storage==2.19.0 --quiet
# fsspec==2024.10.0 as alt. to gcsfs==2025.2.0

# pycairo library has to be installed!
!apt install -y libcairo2-dev

!pip install typeguard==4.4.2 pycairo==1.27.0 --quiet
# stupid torchtyping requires medival version typeguard==2.13.3
# ..wont be resolved.
!echo -= Done =-


Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab-4-Studienarbeit
/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI
-= Install dependencies =-
/bin/bash: line 1: 2.6: No such file or directory
-= Resolve other dependency issue =-
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtyping 0.1.5 requires typeguard<3,>=2.11.1, but you have typeguard 4.4.2 which is incompatible.
-= Done =-


In [ ]:
#@title Install ComfyUI-Manager
WORKSPACE = "/content/drive/MyDrive/Colab-4-Studienarbeit/" #.@param {type:"string"}
UPDATE_COMFY_UI_MANAGER = False  #.@param {type:"boolean"}

comfyui_custom_nodes = os.path.join(WORKSPACE, "ComfyUI", "custom_nodes")
![ ! -d $comfyui_custom_nodes ] && echo -= No ComfyUI-Installation found! =- && exit 1

%cd $comfyui_custom_nodes

comfyui_manager_path = os.path.join(comfyui_custom_nodes, "comfyui-manager")

if not os.path.exists(comfyui_manager_path):
  !git clone https://github.com/ltdrdata/ComfyUI-Manager comfyui-manager
  !echo -= installed ComfyUI-Manager =-
elif UPDATE_COMFY_UI_MANAGER:
  !echo -= updating ComfyUI-Manager =-
  %cd $comfyui_manager_path
  !git pull



/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI/custom_nodes
fatal: destination path 'comfyui-manager' already exists and is not an empty directory.


In [ ]:
#@title Install ComfyUI-3D-Pack (TODO)
# Pre-Builds from https://github.com/MrForExample/Comfy3D_Pre_Builds/tree/main/_Build_Wheels
# ..should be available.. (otherwise there might be issues during build)

COMFYUI_WORKSPACE = "/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI/" #@param {type:"string"}
UPDATE_COMFY_UI_3D_PACK = True  #@param {type:"boolean"}

comfyui_custom_nodes = os.path.join(COMFYUI_WORKSPACE, "custom_nodes")
![ ! -d $comfyui_custom_nodes ] && echo -= No ComfyUI-Installation found! =- && exit 1

!echo -= Cloning ComfyUI-3D-Pack =-
comfyui_3dpackdir = os.path.join(comfyui_custom_nodes, "ComfyUI-3D-Pack")

if os.path.exists(comfyui_3dpackdir):
  !echo -= ComfyUI-3D-Pack already installed =-
  %cd $comfyui_3dpackdir
  if UPDATE_COMFY_UI_3D_PACK:
    !echo -= Updating ComfyUI-3D-Pack.. =-
    !git pull
else:
  !echo -= Installing ComfyUI-3D-Pack =-
  !git clone https://github.com/MrForExample/ComfyUI-3D-Pack.git
  UPDATE_COMFY_UI_3D_PACK = True

%cd $comfyui_custom_nodes

if UPDATE_COMFY_UI_3D_PACK:
  !echo -= Creating Venv for pre-build generation =-
  !apt install gcc g++
  %cd "/content/"
  !apt install python3.11-venv
  !python -m venv comfyui3dPackBuildenv
  # !source "/content/comfyui3dpack_buildenv/bin/activate"
  # ⚠️ you canot source it like that, call executables explicitly instead.
  !echo -= pulling current version of ComfyUI-3D-Pack =-
  %cd $comfyui_3dpackdir

  !echo -= Install dependencies =-
  !/content/comfyui3dpack_buildenv/bin/pip install -r requirements.txt --quiet

  !echo -= Install Pre_Build dependencies =-
  !/content/comfyui3dpack_buildenv/bin/python install.py
  !deactivate
  # ..comfyui3dpack_buildenv



-= Cloning ComfyUI-3D-Pack =-
-= ComfyUI-3D-Pack already installed =-
/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI/custom_nodes
-= Creating Venv for pre-build generation =-
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
gcc is already the newest version (4:11.2.0-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.11-venv is already the newest version (3.11.11-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
/bin/bash: line 1: /content/comfyui3dpack_buildenv/bin/activate: No such file or directory
-= pulling current version of ComfyUI-3D-Pack =-
/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI/custom_nodes/ComfyUI-3D-Pack
Already up to date.
-= Install dependencies =-
  Installing build dependencies ... done
  

In [ ]:
#@title Initialize Model Downloader

import os
# ? for path handling and symlink creation.
from dataclasses import dataclass
from typing import List, Optional

CUI_MODEL_STORE = "/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI/models" #@param {type:"string"}
TMP_MODEL_STORE = "/content/temporaryModelStore/" #@param {type:"string"}
# ? Store where actual model-files will be stored.
#   ..bcs. I dont want those big files on my g-drive and rather download them for each session.
# ? if TMP_MODEL_STORE is None or TMP_MODEL_STORE == CUI_MODEL_STORE:
#       it will be stored in CUI_MODEL_STORE with relative_local_dir as suffix.
#        => *is required by ComfyUI due to different model types.*

if not os.path.exists(CUI_MODEL_STORE):
  raise ValueError(f"No ComfyUI-Installation found in '{CUI_MODEL_STORE}'!\n Did you mount your drive or installed ComfyUI properly or did you provide a valuty path?")

os.makedirs(TMP_MODEL_STORE, exist_ok=True)

@dataclass
class HuggingFaceFileDownloader:
    repository: str  # just repo-name seems to be enough..
    files: List[str] # with repo-root as base.
    fileRenames: Optional[str] = None
    # ? fileRenames on links are only applied on such, **not the source itself**!
    # ? on the other hand, if no link is required source will be renamed,
    #   as you expect it to fullfill following dependencies.
    subfolder: Optional[str] = None
    relative_local_dir: str = ""  # relative to CUI_MODEL_STORE

hf_download_que: List[HuggingFaceFileDownloader] = []
# ? you have to append different HuggingFaceFileDownloader to this list
# ? and run `Fetch previously qued Models for installation` to install them in parallel..


-= =-


ValueError: No ComfyUI-Installation found in '/content/drive/MyDrive/Colab-4-Studienarbeit/ComfyUI/models'!
 Did you mount your drive or installed if properly or provide a valuty path?

In [ ]:
#@title Install Unique3D models [optional]

# Unique3D - Multiview Image reconstruction (Triplane?)
# Installing five models according to..
!echo -= quing Checkpoint Model =-
hf_download_que.append(
  HuggingFaceFileDownloader(
    repository="fangjiaqi/SDV1.5-pruned-emaonly-offical",
    files=["v1-5-pruned-emaonly.ckpt"],
    relative_local_dir="checkpoints"
  )
)

!echo -= quing Controlnet Model =-
hf_download_que.append(
  HuggingFaceFileDownloader(
    repository="Wuvin/Unique3D",
    files=["diffusion_pytorch_model.safetensors"],
    relative_local_dir="controlnet"
  )
)

!mkdir -p $CUI_MODEL_STORE"/ipadapter"
# ? as it is not present by default.
!echo -= quing Ipadapter Model =-
hf_download_que.append(
  HuggingFaceFileDownloader(
    repository="h94/IP-Adapter",
    subfolder="models/",
    files=["ip-adapter_sd15.safetensors"],
    relative_local_dir="ipadapter"
  )
)

!echo -= quing Clip-Vision Model =-
hf_download_que.append(
  HuggingFaceFileDownloader(
    repository="h94/IP-Adapter",
    subfolder="models/image_encoder/",
    files=["model.safetensors"],
    fileRenames=["OpenCLIP-ViT-H-14.safetensors"],
    relative_local_dir="clip_vision"
  )
)

!echo -= downloading Real-ESRGAN Upscale-Model from Github =-
upscale_model_dir = os.path.join(CUI_MODEL_STORE, "upscale_models")
upscale_model_path = os.path.join(upscale_model_dir, "RealESRGAN_x4plus.pth")
if not os.path.exists(upscale_model_path):
    # is fine to be stored on drive, as it is small.
    !wget -P {upscale_model_dir} https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth


In [ ]:
#@title Additional Models (TODO)

# TripoSR - NeRF & marching cube hybrid
# StableFast3D - single image to 3D Mesh
# Triplane Gaussian Transformers - fast 3D Gaussian
# MS Trellis
# *https://github.com/microsoft/TRELLIS*

In [ ]:
#@title Fetch previously qued Models for installation

import multiprocessing
from huggingface_hub import hf_hub_download
from google.colab import userdata
HF_API_TOKEN = userdata.get('huggingface-api-token')


def download_files_from_huggingface(
    downloader: HuggingFaceFileDownloader,
    huggingface_api_token: str,
    model_store: str
  ):
    model_path = os.path.join(model_store, downloader.relative_local_dir)
    os.makedirs(model_path, exist_ok=True)

    for filename in downloader.files:
      # Do not download if already present.
      targetFile = os.path.join(model_path, filename)
      if os.path.exists(targetFile):
          !echo -= {targetFile} is already present! Will not be fetched again. =-
          continue

      hf_hub_download(
        repo_id=downloader.repository,
        filename=filename,
        local_dir=model_path,
        token=huggingface_api_token  # should disable throttle
      )

def apply_file_renames(downloader: HuggingFaceFileDownloader, model_store: str):
  if downloader.fileRenames.count == 0: return
  model_path = os.path.join(model_store, downloader.relative_local_dir)

  for i in range(len(downloader.fileRenames)):
    if downloader.files[i] is None: return
    oldFile = os.path.join(model_path, downloader.files[i])
    newFile = os.path.join(model_path, downloader.fileRenames[i])
    os.rename(oldFile, newFile)
# ..or..
def create_file_links(downloader: HuggingFaceFileDownloader, model_store: str, target_model_store: str):
  if model_store == target_model_store or target_model_store is None:
    return False # as no link creation required.
  # model_store can be None, it will just work thorough current dir.

  # will attend to correct naming per downloader.fileRenames
  for i in range(len(downloader.files)):
    srcFile = os.path.join(model_store, downloader.files[i])  # sources should never change.

    if downloader.fileRenames[i] is None:
      destination_store = model_store
    else:
      # apply "renaming" only to destination.
      destination_store = target_model_store

    destinationFile = os.path.join(
        destination_store,
        downloader.relative_local_dir,
        downloader.files[i]
      )
    os.link(srcFile, destinationFile)

  return True

#
# Main
#

if len(hf_download_que) == 0:
  raise "Did you forget to actually que hugging-face downloads?"

if TMP_MODEL_STORE is None or not isinstance(TMP_MODEL_STORE, str):
  model_store = CUI_MODEL_STORE
else:
  model_store = TMP_MODEL_STORE

!echo -= asynchroniously download all qued HuggingFace-Models =-
with multiprocessing.Pool(processes=len(hf_download_que)) as pool:
    pool.starmap(
        download_files_from_huggingface,
        [(downloader, HF_API_TOKEN, model_store) for downloader in hf_download_que]
    )
    pool.close()  # everything qued
    pool.join()  # wait for finis..

!echo -= Done downloading HuggingFace-Models. =-

!echo -= Performing Post-Functions.. =-
# Create symlinks to target or rename inplace, as needed.
for downloader in hf_download_que:
  # primarily create file links, as it is default.
  if not create_file_links(downloader, CUI_MODEL_STORE, TMP_MODEL_STORE):
    apply_file_renames(downloader, model_store)
    !echo -= Did rename all files in model_store they have been fetched to =-
  else:
    !echo -= Did create Hardlinks for fetched huggingface-files =-

# Cleanup Que (for possible reque in current session)
hf_download_que.clear()

In [ ]:

#@title Run ComfyUI via cloudflared
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!rm -rf cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request
# I had to add..
from google.colab import userdata
CF_TOKEN = userdata.get('cloudflared-tunnel-token')
!cloudflared service install $CF_TOKEN


def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    print(l, end='')


# threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server


--2025-03-05 16:59:50--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb [following]
--2025-03-05 16:59:50--  https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/4f284b8f-4452-49e8-a0c0-0e45c3b6e65e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250305%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250305T165950Z&X-Amz-Expires=300&X-Amz-Signature=5c49a95341eb171daa3e9d43c0264ad1c351ff9cfa170f682ead39801742